In [6]:
from bitarray import bitarray
from functools import reduce

class Register:
    def __init__(self, reg_len, seed, sync : int, feedback_cells : list):
        '''
        reg_len (int) : длина регистра
        seed (str) : вектор инициализации
        sync (int): индекс бита синхронизации (слева-направо с единицы)
        feedback_cells (list) : список индексов битов отводной последовательности (слева-направо с единицы)
        '''
        ba = bitarray()
        ba.frombytes(seed.encode('utf-8'))
        ba += bitarray('0') * (reg_len - len(ba))
        self.register = ba
        self.sync_cell = sync - 1
        self.feedback_cells = list(map(lambda x: x - 1, feedback_cells))
    def feedback(self):
        return reduce(lambda x, y: x ^ y, [self.register[i] for i in self.feedback_cells])
    def sync(self):
        return self.register[self.sync_cell]
    def bit(self, sync_all):
        bit = self.register[0]
        if self.sync() == sync_all:
            feedback = self.feedback()
            self.register <<= 1
            self.register[-1] = feedback
        return bit

In [21]:
def tact(x, y, z):
    x, y, z = x.sync(), y.sync(), z.sync()
    return ((x and y) or (x and z) or (y and z))

def a5_1():
    registers = [
        Register(19, seed='abcd', sync=11, feedback_cells=[0,1,2,5]),
        Register(22, seed='ikj', sync=12, feedback_cells=[0,1]),
        Register(23, seed='k', sync=13, feedback_cells=[0,1,2,16]),
    ]
    for r in registers:
        print(r.register)
    while True:
        sync_all = tact(*registers)
        yield reduce(lambda x, y: x ^ y, map(lambda x: x.bit(sync_all), registers))

In [137]:
plaintext = 'НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК'
ba = bitarray()
ba.frombytes(plaintext.encode('utf-8'))
gamma_len = len(ba)
gamma_len

672

In [144]:
gamma_len // 8, len(plaintext.encode('utf-8')), len(plaintext) * 2

(84, 84, 84)

In [19]:
def a5_1_enc(plaintext):
    plain_bits = bitarray()
    plain_bits.frombytes(plaintext.encode('utf-8'))
    cipher_bits = bitarray()

    for i in a5_1():
        if plain_bits:
            bit = plain_bits.pop()
            #print(*[bit, i])
            cipher_bits.append(bit ^ i)
        else:
            break
            
    cipher_bits.reverse()
    return cipher_bits.tobytes()

In [22]:
a5_1_enc('НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК')

bitarray('01100001011000100110001101100100')
bitarray('011010010110101101101010')
bitarray('01101011000000000000000')


b'V\x92\xc1\xb5\x13R\x9a\x15\x17\x04?\xad\xaf\t,\xd8l\x19.\x84M\xdf[m\xf8\xd8\r\x038\xae\x95c8\xf3\xffY\x88@\x87\xab\xad\xe18+\xb1\x8a\xa9\xb4\xe2K\xbb\xff\x06E\xe0\xae\xdb\xa0\x99lT\xc9\xd5Q\x80\xc7i\xac\x17\xf4{\x1a\xb0\x9e\x8e\xee\xd44\x98,<\x8e\xbf>'

In [9]:
def a5_1_dec(ciphertext):
    plain_bits = bitarray()
    cipher_bits = bitarray()
    cipher_bits.frombytes(ciphertext)

    for i in a5_1():
        if cipher_bits:
            plain_bits.append(cipher_bits.pop() ^ i)
        else:
            break
            
    plain_bits.reverse()
    return plain_bits.tobytes().decode('utf-8')

In [203]:
a5_1_dec(a5_1_enc('НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК'))

'НЕВСЯКИЙЗПТКТОЧИТАЕТЗПТВЧТЕНИИСИЛУЗНАЕТТЧК'

In [1]:
import requests
from bs4 import BeautifulSoup as bs

def clear_paragraphs(ps:list, start=14, stop=16):
    return ''.join([i.text for i in ps[start:stop]]).replace('\xad', '').replace('\xa0', '')

def test_text(url='https://spacemorgue.com/matter-matters/'):
    '''
    Возвращает тестовый текст длиной 2408 символов
    '''
    soup = bs(requests.get(url).text, 'html.parser')
    return clear_paragraphs(soup.select_one('article').find_all('p'))

def prepare(text, specials={',':'ЗПТ','.':'ТЧК', ' ':'', '—':'ТИРЕ', '«':'КВЧ', '»':'КВЧ'}):
    '''
    Возвращает текст, удаляя пробельные символы в тексте и заменяя пунктуацию телеграфными обозначениями
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

In [2]:
text = test_text()

In [10]:
a5_1_enc(text)[:150]

b'U \xe8\xcde\xfd[cW\xb6\xe6f\x95\x03\xd7\xcc\x8d\xd8\x1f\x7f\xa5\x03\x99\xc7\xae?\x97\xda?d\xbdx\x8f\xa2Vl\x98vd\x97\x0fK\xec\x86#\xd8\xa5\x1c0\x0bt\x94\xeb-\x05\xea\xf7\xbbfG\xf3\x1b\xc7W\xaftJJ\xee\xcdw\x98\xa3x)\xa6\x1d\x84\xe8Q5j\x01\x95P7\xe2\xbe\xe5\xe9f\x98\xec\x7f\xd9\xfd`Zz^\x0e\xb9\x0e\x11\xde\xa8c9\xcf\xadS\x04pZhNX6T\x97\x0e%\x02\xd9\x90\x81<\xc9\xdc\xfa\xed\xb9G\xa0\x84\xf9\x95\x03)\x9fI\x80i\xe3\xbc\xf5,t\xdbL'

In [11]:
a5_1_dec(a5_1_enc(text))[:150]

'Нелинейная и статистическая причинность снова привносят в философскую концепцию каузальной связи сложность, которая была потеряна с появлением понятия'